# GeoLlama: Fine-Tuning on Retrieval Augmented Geoparsing
This notebook wlaks through the process of producing the `GeoLlama_RAG` model used to geoparse text. We will be using the LGL and GeoVirus datasets to fine-tune the model. These have been pre-processed and combined into the `llama3_RAG_geoparsing_ft.json` dataset.

We will use Unsloth to optimize the fine tuning process. Thi will make for fater, more memory efficient training, alhtough it does mean the training can only be done on a Linux machine. A Google Colab GPU instance is optimal for this.

Fine tuning require ~20GB of GPU RAM, depending on GPU model.

In [1]:
# standard library imports
import os
import json
from ast import literal_eval
# third party imports
from google.colab import drive
from unsloth import FastLanguageModel
from huggingface_hub import login
from geopy.distance import distance
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import numpy as np
import torch
#local imports
from geo_llama.model import RAGModel

drive.mount('/content/drive')

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



## 1. Setting up the Unsloth package
The first step is to initilise the Unsloth package used for fine-tuning. We're using some of the out-of-the-box parameter settings for this.

In [2]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)



In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # 4x longer contexts auto supported!
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)



Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## 2. Data Pre-processing
We will use the `llama3_RAG_geoparsing_ft.json` dataset to fine-tune and test the model. The dataset has 3983 examples, which isn't a huge number, but the relative simplicty of the task and the requirement for a highly structured output means we don't need an extremely large training set.

We'll use 3600 samples from the dataset as training data and use the remaining 383 samples to test the model.

Note that each sample from this dataset represents a piece of text, a single toponym from that text, and a list of potential locations matching that toponym found via Nominatim. Hence, the 817 samples from the toponym extraction dataset become 3983 samples in this dataset, since we have a single sample for each inidividual toponym in the text.

In [3]:
with open('data/fine_tuning_data/llama3_RAG_geoparsing_ft.json') as f:
    data = json.load(f)

In [4]:
len(data)

3983

In [5]:


RAG_prompt = """Below is an instruction that describes a task, paired with an input that provides a specfic example which the task should be applied to. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}
"""


In [ ]:

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["response"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = RAG_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

ft_data = {"items":data}

In [4]:
from datasets import Dataset
import pandas as pd
import random

random.seed(7723)
trn_idxs = random.sample(range(len(data)), 3600)
val_idxs = [x for x in range(len(data)) if x not in trn_idxs]
trn_data = [data[i] for i in trn_idxs]
val_data = [data[i] for i in val_idxs]

trn_dataset = Dataset.from_pandas(pd.DataFrame(trn_data), split="train")

In [ ]:
trn_dataset = trn_dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/3600 [00:00<?, ? examples/s]

## 3. Training
This section shows the process of model training. Training takes ~100 minutes on a T4 GPU. We will use the `SFTTrainer` class to train the model, setting the parameters as shown below.

In [ ]:

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = trn_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        num_train_epochs = 1,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Map (num_proc=2):   0%|          | 0/3600 [00:00<?, ? examples/s]

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA L4. Max memory = 22.168 GB.
5.594 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,600 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 450
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.885000
2,1.996400
3,1.748000
4,1.777100
5,1.684900
6,1.441200


Step,Training Loss
1,1.885000
2,1.996400
3,1.748000
4,1.777100
5,1.684900
6,1.441200
7,1.663200
8,1.586100
9,1.328600
10,1.498900


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")



6639.6193 seconds used for training.
110.66 minutes used for training.
Peak reserved memory = 8.967 GB.
Peak reserved memory for training = 3.373 GB.
Peak reserved memory % of max memory = 40.45 %.
Peak reserved memory for training % of max memory = 15.216 %.


In [11]:
# save the model to hugginface
login()

In [ ]:
#model.save_pretrained("GeoLlama.pt")
model.push_to_hub("JoeShingleton/GeoLlama_7b_RAG")

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/JoeShingleton/GeoLlama_7b_RAG


## 4. Testing the model
We will now test the model with our 67 testing samples. We'll use the `RAGModel` class form the `geo_llama` package as a wrapper for the fine-tuned model. This module will handle cleaning up any responses and processing inputs.

We'll assess the accuracy as the proportion of unique toponyms form the text which have been found by the model. Note: we are not expecting the model to identify the location of the toponym within the text, so the overlap will not be assessed.

In [6]:
from geo_llama.model import RAGModel

# set up the model
model = RAGModel(model_name='JoeShingleton/GeoLlama_7b_RAG',
                  prompt_path='data/prompt_templates/prompt_template.txt',
                  instruct_path='data/prompt_templates/topo_instruction.txt',
                  input_path='data/prompt_templates/rag_input.txt',
                  config_path='data/config_files/model_config.json')

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.25.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [8]:
d['input'].split('<matches>')[1][:-12].replace("'", '"')

' [{"name": "Central Business District", "lat": "-1.2832533", "lon": "36.8172449", "address": "Central Business District, CBD division, Starehe, Nairobi, 46464, Kenya"}, {"name": "Nairobi", "lat": "-1.3026148499999999", "lon": "36.82884201813725", "address": "Nairobi, Starehe, Nairobi, Kenya"}, {"name": "Nairobi", "lat": "-1.3026148499999999", "lon": "36.82884201813725", "address": "Nairobi, Kenya"}, {"name": "Nairobi", "lat": "-1.3026148499999999", "lon": "36.82884201813725", "address": "Nairobi, Kenya"}, {"name": "Nairobi", "lat": "-0.0363549", "lon": "36.1205614", "address": "Nairobi, Solai ward, Rongai, Nakuru, Rift Valley, Kenya"}, {"name": "Nairobi", "lat": "-0.1628443", "lon": "35.6753805", "address": "Nairobi, Kamara ward, Kuresoi North, Nakuru, Rift Valley, Kenya"}, {"name": "Nairobi", "lat": "-0.0210221", "lon": "36.2344774", "address": "Nairobi, Subukia ward, Subukia, Nakuru, Rift Valley, Kenya"}, {"name": "Mto Nairobi", "lat": "-1.2762591", "lon": "36.8175002", "address": "

In [10]:
from tqdm import tqdm

outputs = []
for d in tqdm(val_data):
  prompt = RAG_prompt.format(d['instruction'], d['input'], '')
  matches = literal_eval(d['input'].split('<matches>')[1][:-12])
  tst_outputs = model.get_output(prompt, matches)
  outputs.append(tst_outputs)


  0%|          | 1/383 [00:03<20:07,  3.16s/it]



{'name': 'Grove City', 'latitude': '39.8814519', 'longitude': '-83.0929645', 'RAG_estimated': True}



  1%|          | 2/383 [00:07<24:56,  3.93s/it]



{'name': 'SPRINGFIELD', 'latitude': '39.9234046', 'longitude': '-83.810138', 'RAG_estimated': True}



  1%|          | 3/383 [00:11<24:02,  3.80s/it]

...



{'name': 'Toledo', 'latitude': '41.6529143', 'longitude': '-83.5378173', 'RAG_estimated': True}



100%|█████████▉| 382/383 [22:50<00:03,  3.59s/it]



{'name': 'United States', 'latitude': '39.7837304', 'longitude': '-100.445882', 'RAG_estimated': True}



100%|██████████| 383/383 [22:53<00:00,  3.59s/it]



{'name': 'Yogyakarta', 'latitude': '-7.8011998', 'longitude': '110.3646608', 'RAG_estimated': True}



In [13]:
distances = []
for i, (true, pred) in enumerate(zip(val_data, outputs)):
  true_dict = json.loads(true['response'].replace("'", '"').replace('True', 'true').replace('False', 'false'))
  true_lat = true_dict['latitude']
  true_lon = true_dict['longitude']
  pred_lat = pred['latitude']
  pred_lon = pred['longitude']
  try:
    distances.append(distance((true_lat, true_lon), (pred_lat, pred_lon)).km)
  except:
    distances.append(distance((true_lat, true_lon), (pred_lon, pred_lat)).km)


In [14]:

print(f'Mean distance : {np.mean(distances)}')
print(f'Median distance : {np.median(distances)}')
print(f'Accuracy @ 1km : {len([d for d in distances if d < 1])/len(distances)}')
print(f'Accuracy @ 10km : {len([d for d in distances if d < 10])/len(distances)}')
print(f'Accuracy @ 50km : {len([d for d in distances if d < 50])/len(distances)}')
print(f'Accuracy @ 161km : {len([d for d in distances if d < 161])/len(distances)}')

Mean distance : 210.14376934563265
Median distance : 0.0
Accuracy @ 1km : 0.8276762402088773
Accuracy @ 10km : 0.8511749347258486
Accuracy @ 50km : 0.8746736292428199
Accuracy @ 161km : 0.8903394255874674
